In [1]:
def load_csv(filename):
    with open(filename, 'r', encoding='utf-8-sig') as f:
        csv_list = [row.strip().split(',') for row in f.readlines()]
    return csv_list

def printx(listx):
    for x in listx:
        print(x)

In [2]:
def parse_row(row):
    name = row[0].lower()
    prevElemCombos = row[1].lower().split(' / ')
    prevElemCombos = [p.lower().split('+') for p in prevElemCombos]
    if prevElemCombos == [['available from start.']]:
        prevElemCombos = [['BASE_ELEMENT']]
    prevElemCombos.sort()
    for p in prevElemCombos:
        p.sort()
    return [prevElemCombo + [name] for prevElemCombo in prevElemCombos]

def parse_rows(csv_list):
    r = []
    for row in csv_list:
        r.extend(parse_row(row))
    return r

In [3]:
elemsFilename = 'elements_data.csv'
csv_list = load_csv(elemsFilename)

In [4]:
import pandas as pd
from tqdm import tqdm

def create_elements_df(csv_list):
    r = parse_rows(csv_list)
    df = pd.DataFrame(r, columns=['elem1','elem2', 'result'])
    base_elements = get_base_elements(csv_list)
    return df[~df.index.isin(base_elements.index)]


def get_base_elements(csv_list):
    df = pd.DataFrame(parse_rows(csv_list), columns=['elem1','elem2', 'result'])
    return df[df['elem1']=='BASE_ELEMENT']

def get_elements_list(dfs):
    r = []
    for col in list(dfs.columns):
        r.extend(dfs[col].tolist())
    return list(set(r))


In [5]:
dfs = create_elements_df(csv_list)
dfs.head()

,elem1,elem2,result
0,rain,smog,acid rain
1,rain,smoke,acid rain
3,bird,metal,airplane
4,bird,steel,airplane
5,clock,sound,alarm clock


In [6]:
base_elements = get_base_elements(csv_list)['elem2'].tolist()
print(base_elements)

['air', 'earth', 'fire', 'water']


In [7]:
def get_unique_current_combos(curr_elements, dfs):
    possible_combos = [[elem1, elem2] for elem1 in curr_elements for elem2 in curr_elements if elem1 != elem2]
    for i in range(len(possible_combos)):
        possible_combos[i].sort()
        possible_combos[i] = '+'.join(possible_combos[i])
    possible_combos = list(set(possible_combos))
    possible_combos = [p.split('+') for p in possible_combos]
    return possible_combos

def get_elem_score(elem, dfs, levels=0):
    dFilter = (dfs['elem1']==elem) | (dfs['elem2']==elem)
    if levels == 0:
        return sum(dFilter)
    else:
        next_possible_elements = dfs.loc[dFilter, 'result'].tolist()
        return sum([get_elem_score(e, dfs, levels-1) for e in next_possible_elements])

def get_next_possible_combos(curr_elements, dfs):
    r = []
    unique_combos = get_unique_current_combos(curr_elements, dfs)
    for combo in unique_combos:
        l = (dfs['elem1']==combo[0]) & (dfs['elem2']==combo[1])
        if l.any():
            r.append([dfs.loc[l, 'elem1'].tolist()[0], dfs.loc[l, 'elem2'].tolist()[0] , dfs.loc[l, 'result'].tolist()[0]])
    return [x for x in r if x[-1] not in curr_elements]

def get_next_possible_elements(curr_elements, dfs):
    r = get_next_possible_combos(curr_elements, dfs)
    return [x[-1] for x in r]


In [8]:

def get_next_best_move(curr_elements, dfs, levels=0):
    next_possible_combos = get_next_possible_combos(curr_elements, dfs)
    scores = [get_elem_score(e[-1], dfs, levels) for e in next_possible_combos]
    return next_possible_combos[scores.index(max(scores))]

def fullCombo_to_string(combo):
    return ' + '.join(combo[:-1]) + ' = ' + combo[-1]

In [9]:
def get_n_moves(curr_elements, dfs, levels=0, n=10, print_moves=False):
    curr_elements = list(curr_elements)
    r = []
    for i in (range(n) if print_moves else tqdm(range(n))):
        newCombo = get_next_best_move(curr_elements, dfs, levels=levels)
        r.append(fullCombo_to_string(newCombo))
        if print_moves:
            print(i+1, fullCombo_to_string(newCombo))
        curr_elements.append(newCombo[-1])
        curr_elements.sort()
    return r

# **Complete example**


In [10]:
elemsFilename = 'elements_data.csv'
csv_list = load_csv(elemsFilename)

dfs = create_elements_df(csv_list)
base_elements = get_base_elements(csv_list)['elem2'].tolist()
nx = 25
x = get_n_moves(curr_elements=base_elements, dfs=dfs, levels=0, n=nx, print_moves=True)

1 air + fire = energy
2 air + energy = wind
3 air + water = rain
4 earth + rain = plant
5 earth + plant = grass
6 earth + fire = lava
7 air + lava = stone
8 fire + stone = metal
9 energy + metal = electricity
10 air + stone = sand
11 electricity + sand = glass
12 metal + stone = blade
13 glass + metal = glasses
14 electricity + metal = wire
15 blade + metal = sword
16 earth + water = mud
17 mud + sand = clay
18 grass + mud = swamp
19 energy + swamp = life
20 earth + life = human
21 air + life = bird
22 human + metal = tool
23 life + stone = egg
24 bird + metal = airplane
25 human + plant = farmer


In [11]:

x = get_n_moves(curr_elements=base_elements, dfs=dfs, levels=1, n=nx, print_moves=True)

1 air + fire = energy
2 air + water = rain
3 earth + rain = plant
4 earth + plant = grass
5 air + energy = wind
6 earth + water = mud
7 energy + water = steam
8 air + steam = cloud
9 air + cloud = sky
10 fire + sky = sun
11 grass + mud = swamp
12 energy + swamp = life
13 earth + life = human
14 human + plant = farmer
15 farmer + life = livestock
16 air + life = bird
17 grass + livestock = cow
18 human + rain = cold
19 earth + fire = lava
20 air + lava = stone
21 fire + stone = metal
22 human + metal = tool
23 air + stone = sand
24 life + stone = egg
25 energy + metal = electricity


In [12]:
x = get_n_moves(curr_elements=base_elements, dfs=dfs, levels=2, n=nx, print_moves=True)

1 air + fire = energy
2 air + water = rain
3 earth + rain = plant
4 earth + plant = grass
5 earth + fire = lava
6 air + lava = stone
7 fire + stone = metal
8 air + stone = sand
9 fire + sand = glass
10 energy + metal = electricity
11 earth + water = mud
12 grass + mud = swamp
13 energy + swamp = life
14 earth + life = human
15 human + metal = tool
16 human + plant = farmer
17 life + stone = egg
18 air + egg = bird
19 egg + swamp = lizard
20 air + energy = wind
21 farmer + life = livestock
22 egg + sand = turtle
23 earth + farmer = field
24 energy + water = steam
25 air + steam = cloud


----------------------------------------

In [13]:
x = get_n_moves(curr_elements=base_elements, dfs=dfs, levels=3, n=nx, print_moves=True)

1 air + fire = energy
2 earth + water = mud
3 air + water = rain
4 earth + rain = plant
5 mud + plant = swamp
6 energy + swamp = life
7 earth + life = human
8 human + plant = farmer
9 earth + plant = grass
10 earth + fire = lava
11 air + lava = stone
12 air + stone = sand
13 fire + stone = metal
14 fire + sand = glass
15 human + metal = tool
16 energy + metal = electricity
17 life + stone = egg
18 air + energy = wind
19 earth + farmer = field
20 air + egg = bird
21 energy + water = steam
22 air + steam = cloud
23 air + cloud = sky
24 fire + sky = sun
25 sun + tool = solar cell
